In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import random
import matplotlib.pyplot as plt

In [2]:
#!pip install pyomo

In [3]:
import pandas as pd

In [7]:
num_edges = 82
num_nodes = 48
oper_costs = range(5,11)
num_cnfNumber = 5
num_sfcRequests = 3
end_delays = range(30,51)
cpu_bounds = range(12,15+1)  #(4,6+1)
bandwidth_bounds = range(4,6+1)

In [8]:
EdgeList=[(1,2),
(1,3),
(2,3),
(2,5),
(3,5),
(3,4),
(5,6),
(4,6),
(5,16),
(6,16),
(4,7),
(4,8),
(7,16),
(7,9),
(9,8),
(8,12),
(8,11),
(9,11),
(9,10),
(11,12),
(10,11),
(12,13),
(11,13),
(13,15),
(13,14),
(14,15),
(10,14),
(15,23),
(14,20),
(10,21),
(20,23),
(10,16),
(16,17),
(16,21),
(20,21),
(17,18),
(17,21),
(21,24),
(20,24),
(23,24),
(18,19),
(19,27),
(22,24),
(22,26),
(24,25),
(25,26),
(25,31),
(25,30),
(26,27),
(27,30),
(28,30),
(27,28),
(27,32),
(27,29),
(28,29),
(28,31),
(29,34),
(37,38),
(34,38),
(31,37),
(37,40),
(38,40),
(32,34),
(38,39),
(34,35),
(32,33),
(39,40),
(33,36),
(35,36),
(35,39),
(39,45),
(36,41),
(41,45),
(41,42),
(41,44),
(44,45),
(45,46),
(42,43),
(44,47),
(46,47),
(43,48),
(47,48)]

# EdgeList=[]
# while len(EdgeList)<21:
#     n1 = random.choice(range(0,num_nodes))
#     n2 = random.choice(range(0,num_nodes))
#     if(n1 == n2):
#         continue
#     elif(n1<n2):
#         EdgeList.append((n1,n2))
#     else:
#         if((n2,n1) not in EdgeList):
#             EdgeList.append((n2,n1))

In [33]:
L={(1, 2): 1.5884322210000001,
 (1, 3): 1.366945662,
 (2, 3): 0.5950783458000001,
 (2, 5): 0.6067530891999999,
 (3, 5): 0.42462709319999997,
 (3, 4): 0.6120901147,
 (5, 6): 0.7058216254,
 (4, 6): 0.2038076622,
 (5, 16): 0.9106299799,
 (6, 16): 0.623764858,
 (4, 7): 0.2635156352,
 (4, 8): 0.8185662896,
 (7, 16): 0.6007489355,
 (7, 9): 0.5447101675,
 (9, 8): 0.31888727499999997,
 (8, 12): 0.4252942214,
 (8, 11): 0.3902699914,
 (9, 11): 0.2641827634,
 (9, 10): 0.35524576139999997,
 (11, 12): 0.2204858669,
 (10, 11): 0.2491723791,
 (12, 13): 0.1307571253,
 (11, 13): 0.1010699208,
 (13, 15): 0.09606645942,
 (13, 14): 0.1581093811,
 (14, 15): 0.1217508947,
 (10, 14): 0.3215557878,
 (15, 23): 0.5050160401,
 (14, 20): 0.2892000705,
 (10, 21): 0.3916042478,
 (20, 23): 0.4082824525,
 (10, 16): 0.7635282139,
 (16, 17): 0.8475863658999999,
 (16, 21): 0.7048209332,
 (20, 21): 0.5470451161,
 (17, 18): 0.1981370725,
 (17, 21): 0.6431115755,
 (21, 24): 0.8365787508,
 (20, 24): 0.8766064422,
 (23, 24): 0.6197620889000001,
 (18, 19): 0.255843661,
 (19, 27): 0.49400842500000003,
 (22, 24): 0.1010699208,
 (22, 26): 0.3579142741,
 (24, 25): 0.2218201233,
 (25, 26): 0.28152809630000003,
 (25, 31): 1.2188432040000001,
 (25, 30): 0.2985398652,
 (26, 27): 0.03335640952,
 (27, 30): 0.13909622770000002,
 (28, 30): 0.1734533295,
 (27, 28): 0.13008999710000002,
 (27, 32): 0.8455849813,
 (27, 29): 0.2581786097,
 (28, 29): 0.12308515109999998,
 (28, 31): 0.2538422764,
 (29, 34): 0.4783309125,
 (37, 38): 0.24850525089999997,
 (34, 38): 0.2394990204,
 (31, 37): 0.2194851746,
 (37, 40): 0.5226949371999999,
 (38, 40): 0.5313676036999999,
 (32, 34): 0.472993887,
 (38, 39): 0.6484486011,
 (34, 35): 0.5380388856,
 (32, 33): 0.4056139398,
 (39, 40): 0.8379130070999999,
 (33, 36): 0.8555919042,
 (35, 36): 0.44297311840000003,
 (35, 39): 0.22081943099999998,
 (39, 45): 0.5553842185,
 (36, 41): 0.49334129679999994,
 (41, 45): 0.662124729,
 (41, 42): 0.178790355,
 (41, 44): 0.3949398887,
 (44, 45): 0.4936748609,
 (45, 46): 0.6904776771,
 (42, 43): 0.33456478749999996,
 (44, 47): 0.5687267823000001,
 (46, 47): 0.41995719589999997,
 (43, 48): 2.5284158420000002,
 (47, 48): 2.247221309}

In [10]:
init_cpu = 100
init_bandwidth = 100
init_operationalCost = [ random.choice(oper_costs) for i in range(num_nodes)]
init_endDelay = [random.choice(end_delays) for i in range(num_sfcRequests)]
init_bandwidthReq = [random.choice(bandwidth_bounds) for i in range(num_sfcRequests)]

init_cpuReq = {}
for i in range(num_sfcRequests):
    for j in range(num_cnfNumber):
        init_cpuReq[i,j] = random.choice(cpu_bounds)

# Model Definition

In [11]:
model = pyo.ConcreteModel()

In [12]:
model.set_cnfNumber = pyo.RangeSet(0,num_cnfNumber-1) # j {0,1,2,3,4}
model.set_sfcRequests = pyo.RangeSet(0,num_sfcRequests-1) # i {0,1,2} 

# 14 nodes and 21 edges - NSFNET
model.set_nodes = pyo.RangeSet(0,num_nodes-1) # v (0,1,...,13)
model.set_edges = pyo.RangeSet(0,num_edges-1) # (u,v) (0,1,...,21)

# Variables and Parameters

In [13]:
# L - to be done

In [14]:
model.y = pyo.Var(model.set_sfcRequests, model.set_cnfNumber,model.set_nodes,domain=pyo.Binary)

In [15]:
model.z = pyo.Var(model.set_sfcRequests, model.set_edges, domain=pyo.Binary)

In [16]:
model.C = pyo.Param(model.set_nodes, initialize=init_cpu)

In [17]:
model.B = pyo.Param(model.set_edges, initialize=init_bandwidth)

In [18]:
model.W = pyo.Param(model.set_nodes, initialize=init_operationalCost)

In [19]:
model.l = pyo.Param(model.set_sfcRequests, initialize=init_endDelay)

In [20]:
model.c = pyo.Param(model.set_sfcRequests, model.set_cnfNumber, initialize=init_cpuReq)

In [21]:
model.b = pyo.Param(model.set_sfcRequests, initialize=init_bandwidthReq)

In [22]:
y = model.y
z = model.z
C = model.C
B = model.B
W = model.W
l = model.l
c = model.c
b = model.b

# Constraints

In [23]:
model.cpuConstraint = pyo.ConstraintList()
for v in range(num_nodes):
     model.cpuConstraint.add(
         expr=sum(sum(y[i,j,v]*c[i,j] for j in range(num_cnfNumber)) for i in range(num_sfcRequests)) <= C[v]
     )

In [24]:
model.bandwidthConstraint = pyo.ConstraintList()
for _,e in enumerate(EdgeList):
    model.bandwidthConstraint.add(
        expr=sum(z[i,_]*b[i] for i in range(num_sfcRequests)) <= B[_]
    )

In [25]:
model.delayConstraint = pyo.ConstraintList()
# for _,e in enumerate(EdgeList):
for i in range(num_sfcRequests):
    model.delayConstraint.add(
        expr=sum(z[i,_]*L[EdgeList[_][0],EdgeList[_][1]] for _ in range(num_edges)) <= l[i]
    )

In [26]:
model.placementConstraint = pyo.ConstraintList()
for i in range(num_sfcRequests):
    for j in range(num_cnfNumber):
        model.placementConstraint.add(
            expr=sum(y[i,j,v] for v in range(num_nodes)) == 1
        )

# Objective Function

In [27]:
model.objective = pyo.Objective(
    expr=sum(
            sum(
                sum(y[i,j,v]*c[i,j]*W[v] for v in range(num_nodes))
                for j in range(num_cnfNumber)
            )
        for i in range(num_sfcRequests)
    ), sense=minimize
)

# Solving the Model using IBM Cplex

In [28]:
optimizer = SolverFactory('cplex')

In [29]:
optimizer.solve(model)

{'Problem': [{'Name': 'tmpwlxumu9u', 'Lower bound': 1000.0, 'Upper bound': 1000.0, 'Number of objectives': 1, 'Number of constraints': 149, 'Number of variables': 967, 'Number of nonzeros': 1933, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': 0.59, 'Termination condition': 'optimal', 'Termination message': 'MIP - Integer optimal solution\\x3a Objective = 1.0000000000e+03', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 2.9033288955688477}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

# Printing the outputs

In [30]:
Oper_cost = sum(
                sum(
                    sum(y[i,j,v]*c[i,j]*W[v] for v in range(num_nodes))
                    for j in range(num_cnfNumber)
                )
            for i in range(num_sfcRequests)
    )

In [31]:
print("Operational Cost of the current setting: ",pyo.value(Oper_cost))

Operational Cost of the current setting:  1000.0


In [32]:
for i in range(num_sfcRequests):
    for j in range(num_cnfNumber):
        for v in range(num_nodes):
            if(pyo.value(y[i,j,v])):
                print("SFC Number: ",i,",CNF Number: ",j,",Node => ",v)

SFC Number:  0 ,CNF Number:  0 ,Node =>  46
SFC Number:  0 ,CNF Number:  1 ,Node =>  33
SFC Number:  0 ,CNF Number:  2 ,Node =>  33
SFC Number:  0 ,CNF Number:  3 ,Node =>  46
SFC Number:  0 ,CNF Number:  4 ,Node =>  46
SFC Number:  1 ,CNF Number:  0 ,Node =>  46
SFC Number:  1 ,CNF Number:  1 ,Node =>  46
SFC Number:  1 ,CNF Number:  2 ,Node =>  46
SFC Number:  1 ,CNF Number:  3 ,Node =>  33
SFC Number:  1 ,CNF Number:  4 ,Node =>  28
SFC Number:  2 ,CNF Number:  0 ,Node =>  33
SFC Number:  2 ,CNF Number:  1 ,Node =>  33
SFC Number:  2 ,CNF Number:  2 ,Node =>  33
SFC Number:  2 ,CNF Number:  3 ,Node =>  46
SFC Number:  2 ,CNF Number:  4 ,Node =>  33


In [ ]:
for i in range(num_sfcRequests):
    for e in range(num_edges):
        if(pyo.value(z[i,e])):
            print(i,e)

In [ ]:
# EdgeList

# For Variable Number of SFC Requests

In [ ]:
num_edges = 21
num_nodes = 14
oper_costs = range(5,11)
num_cnfNumber = 4
num_sfcRequests = 21
end_delays = range(30,51)
cpu_bounds = range(10,15+1)           #(4,6+1)
bandwidth_bounds = range(4,6+1)
cnfCPU = {1 : 14.32 , 2 : 15.25, 3 : 13.87, 4 : 13.27, 0: 14}

EdgeList=[]
while len(EdgeList)<21:
    n1 = random.choice(range(0,num_nodes))
    n2 = random.choice(range(0,num_nodes))
    if(n1 == n2):
        continue
    elif(n1<n2):
        EdgeList.append((n1,n2))
    else:
        if((n2,n1) not in EdgeList):
            EdgeList.append((n2,n1))

L = {}
for (a,b) in EdgeList:
    L[a,b] = random.choice(range(1,8))

init_cpu = 100
init_bandwidth = 100
# init_operationalCost = [ random.choice(oper_costs) for i in range(num_nodes)]
init_operationalCost = [10, 5, 8, 5, 6, 8, 10, 6, 9, 5, 7, 8, 5, 9]
init_endDelay = [random.choice(end_delays) for i in range(num_sfcRequests)]
init_bandwidthReq = [random.choice(bandwidth_bounds) for i in range(num_sfcRequests)]

init_cpuReq = {}
for i in range(num_sfcRequests):
    for j in range(num_cnfNumber):
        try:
            init_cpuReq[i,j] = cnfCPU[j]/3
        except:
            init_cpuReq[i,j] = random.choice(cpu_bounds)/3

In [ ]:
init_endDelays = init_endDelay
init_bandwidthReqs = init_bandwidthReq 
init_cpuReqs = init_cpuReq

In [ ]:
OperationalCosts = {}

In [ ]:
for num_sfcRequests in reversed(range(1,21)):
    init_endDelay = init_endDelays[0:num_sfcRequests]
    init_bandwidthReq = init_bandwidthReqs[0:num_sfcRequests]
    init_cpuReq = {}
    for i in range(num_sfcRequests):
        for j in range(num_cnfNumber):
            init_cpuReq[i,j] = init_cpuReqs[i,j]
    model = pyo.ConcreteModel()

    model.set_cnfNumber = pyo.RangeSet(0,num_cnfNumber-1) # j {0,1,2,3,4}
    model.set_sfcRequests = pyo.RangeSet(0,num_sfcRequests-1) # i {0,1,2} 

    # 14 nodes and 21 edges - NSFNET
    model.set_nodes = pyo.RangeSet(0,num_nodes-1) # v (0,1,...,13)
    model.set_edges = pyo.RangeSet(0,num_edges-1) # (u,v) (0,1,...,21)

    # Variables and Parameters

    # L - to be done

    model.y = pyo.Var(model.set_sfcRequests, model.set_cnfNumber,model.set_nodes,domain=pyo.Binary)

    model.z = pyo.Var(model.set_sfcRequests, model.set_edges, domain=pyo.Binary)

    model.C = pyo.Param(model.set_nodes, initialize=init_cpu)

    model.B = pyo.Param(model.set_edges, initialize=init_bandwidth)

    model.W = pyo.Param(model.set_nodes, initialize=init_operationalCost)

    model.l = pyo.Param(model.set_sfcRequests, initialize=init_endDelay)

    model.c = pyo.Param(model.set_sfcRequests, model.set_cnfNumber, initialize=init_cpuReq)

    model.b = pyo.Param(model.set_sfcRequests, initialize=init_bandwidthReq)

    y = model.y
    z = model.z
    C = model.C
    B = model.B
    W = model.W
    l = model.l
    c = model.c
    b = model.b

    # Constraints

    model.cpuConstraint = pyo.ConstraintList()
    for v in range(num_nodes):
         model.cpuConstraint.add(
             expr=sum(sum(y[i,j,v]*c[i,j] for j in range(num_cnfNumber)) for i in range(num_sfcRequests)) <= C[v]
         )

    model.bandwidthConstraint = pyo.ConstraintList()
    for _,e in enumerate(EdgeList):
        model.bandwidthConstraint.add(
            expr=sum(z[i,_]*b[i] for i in range(num_sfcRequests)) <= B[_]
        )

    model.delayConstraint = pyo.ConstraintList()
    # for _,e in enumerate(EdgeList):
    for i in range(num_sfcRequests):
        model.delayConstraint.add(
            expr=sum(z[i,_]*L[EdgeList[_][0],EdgeList[_][1]] for _ in range(num_edges)) <= l[i]
        )
        
    model.placementConstraint = pyo.ConstraintList()
    for i in range(num_sfcRequests):
        for j in range(num_cnfNumber):
            model.placementConstraint.add(
                expr=sum(y[i,j,v] for v in range(num_nodes)) == 1
            )

    # Objective Function

    model.objective = pyo.Objective(
        expr=sum(
                sum(
                    sum(y[i,j,v]*c[i,j]*W[v] for v in range(num_nodes))
                    for j in range(num_cnfNumber)
                )
            for i in range(num_sfcRequests)
        ), sense=minimize
    )

    # Solving the Model using IBM Cplex

    optimizer = SolverFactory('cplex')

    optimizer.solve(model)

    # Printing the outputs

    Oper_cost = sum(
                    sum(
                        sum(y[i,j,v]*c[i,j]*W[v] for v in range(num_nodes))
                        for j in range(num_cnfNumber)
                    )
                for i in range(num_sfcRequests)
        )

    print("Operational Cost of the current setting: ",pyo.value(Oper_cost))
    OperationalCosts[num_sfcRequests] = pyo.value(Oper_cost)
    
    
    print("================== ",num_sfcRequests," =================")
    for i in range(num_sfcRequests):
        for j in range(num_cnfNumber):
            for v in range(num_nodes):
                if(pyo.value(y[i,j,v])):
                    print(i,j,v)

In [ ]:
OperationalCosts

In [ ]:
plt.plot(OperationalCosts.keys(),OperationalCosts.values(),'-o')

In [ ]:
plt.show()

In [ ]:
model.pprint()

In [ ]:
# model.param_heatDemand_In_W = pyo.Param(model.set_buildings_BT1, model.set_timeslots, mutable = True, initialize=init_heatDemand)

# model.variable_heatGenerationCoefficient_SpaceHeating = pyo.Var(model.set_buildings_BT1, model.set_timeslots,bounds=(0,1))
# model.variable_temperatureBufferStorage = pyo.Var(model.set_buildings_BT1, model.set_timeslots,  bounds=(0 , 20 ))

In [ ]:
# model = pyo.ConcreteModel()

# model.x = pyo.Var(bounds=(0,10))
# model.y = pyo.Var(bounds=(0,10))
# x = model.x
# y = model.y

# model.C1 = pyo.Constraint(expr= -x+2*y<=8)
# model.C2 = pyo.Constraint(expr= 2*x+y<=14)
# model.C3 = pyo.Constraint(expr= 2*x-y<=10)

# model.obj = pyo.Objective(expr= x+y, sense=maximize)

# opt = SolverFactory('cplex')
# opt.solve(model)

# model.pprint()

# x_value = pyo.value(x)
# y_value = pyo.value(y)

# print('\n---------------------------------------------------------------------')
# print('x=',x_value)
# print('y=',y_value)

In [ ]:
# model = pyo.ConcreteModel()

# model.x = pyo.Var(bounds=(None,3))
# model.y = pyo.Var(bounds=(0,None))
# x = model.x
# y = model.y

# model.C1 = pyo.Constraint(expr = x+y<=8)
# model.C2 = pyo.Constraint(expr = 8*x+3*y>=-24)
# model.C3 = pyo.Constraint(expr = -6*x+8*y<=48)
# model.C4 = pyo.Constraint(expr = 3*x+5*y<=15)

# model.obj = pyo.Objective(expr= -4*x+-2*y, sense=minimize)


In [ ]:
# opt = SolverFactory('cplex')
# opt.solve(model)

# model.pprint()

# x_value = pyo.value(x)
# y_value = pyo.value(y)

# print('\n---------------------------------------------------------------------')
# print('x=',x_value)
# print('y=',y_value)